In [17]:
#pip install requests beautifulsoup4
#!pip install nltk


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
url = "https://www.sluurpy.in/gwalior/restaurants"

response = requests.get(url)

In [4]:
response

<Response [200]>

In [5]:
soup = BeautifulSoup(response.content , 'html.parser')
soup

<!DOCTYPE html>

<html lang="en">
<head>
<link as="font" crossorigin="" href="/fonts/fonts/fontawesome-webfont.woff2?v=4.7.0" rel="preload" type="font/woff2"/>
<link as="style" crossorigin="" href="/fonts/css/font-awesome.min.css" onload="this.rel='stylesheet'" rel="preload"/>
<meta charset="utf-8"/>
<meta content="app-id=1584324158" name="apple-itunes-app"/>
<meta content="app-id=com.sluurpy.social" name="google-play-app"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="Find the best places to eat and restaurants in Gwalior and nearby. View menus and photos, read users' reviews and search by cuisine, price, location and restaurant close to you." name="description"/>
<meta content="The best 10 restaurants and places to eat in Gwalior, October 2023 - Sluurpy" property="og:title">
<meta content="/build/images/logo.a28402f3.png" property="og:image">
<meta content="Find the best places to eat and restaurants in Gwalior and nearby. View menus and photos

In [6]:
data = []

for page_number in range(1, 5): 
    url = f"https://www.sluurpy.in/gwalior/restaurants?page={page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    for heading in soup.find_all('div', class_="main-content"):
        headline = heading.find('h1')
        if headline:
            headline = headline.text.strip()
        else:
            continue

    for restaurant_div in soup.find_all('div', class_="box-ristorante"):
        name = restaurant_div.find('strong')
        if name:
            name = name.text.strip()
        else:
            continue

        ratings_div = restaurant_div.find('div', class_='box-ristorante-voto')
        if ratings_div:
            rating_info = ratings_div.text.strip()
            rating_parts = rating_info.split('(')
            rating = rating_parts[0].strip()
            total_ratings = rating_parts[1].replace('ratings)', '').strip()
        else:
            continue

        tags_container = restaurant_div.find('div', class_='box-ristorante-tags')
        tags = [tag.text for tag in tags_container.find_all('div', class_='box-ristorante-tag')]

        data.append({'headline': headline, 'name': name, 'rating': rating, 'total_ratings': total_ratings, 'tags': tags})


In [7]:
df = pd.DataFrame(data)
df

,headline,name,rating,total_ratings,tags
0,None,Indian Coffee House,78/100,8075,"[Restaurants, Asian, Vegan Options, Indian, Gl..."
1,None,Kwality Restaurant,81/100,4140,"[Restaurants, Asian, Vegan Options, Indian, Ve..."
2,None,Param Food Complex,81/100,3299,"[Restaurants, Fast Food, Indian, Vegetarian Fr..."
3,None,Alfanzo,77/100,3041,"[Bakeries, Dessert, Coffee & Tea, Indian, Chin..."
4,None,Indish,88/100,2798,"[Restaurants, Cafe, Gluten Free Options, Chine..."
...,...,...,...,...,...
93,None,Agan Food Plaza,70/100,32,"[Restaurants, Indian, Indian]"
94,None,R Cube Cafe,70/100,,"[Restaurants, Indian, Indian]"
95,None,Royal Rajput Hotel,70/100,22,"[Restaurants, Middle Eastern, Afghani, Chinese..."
96,None,Baba Ki Kutiaa Restaurant,70/100,7,"[Restaurants, Reservations, Mid-range]"


In [8]:
df.to_csv('restaurant_data.csv', index=False)